# Explore on event level

In [ ]:
import awkward as ak
import uproot
import matplotlib.pyplot as plt

from local_config import raw_path, img_path
from plotting.sum_energy_fit import SumEnergyFit

print(f"{ak.__version__=}")
print(f"{uproot.__version__=}")
print(f"Data file: {raw_path}")

In [ ]:
b_cosmic = uproot.open(raw_path)
print("trees:", b_cosmic.keys())
tree = b_cosmic["ecal"]
print(len(tree))
tree.show()

## Event level info

The following code would be preferable if we would not be loading
the untriggered events afterwards anyways.

```python
events_triggered = tree.arrays(event_level_keys, cut=trigger, entry_stop=entry_stop)
```

In [ ]:
trigger = "nhit_slab > 5"
entry_stop = -1  # Can be reduced for debugging.

event_level_keys = [k for k in tree.keys() if not k.startswith("hit_")]
events_untriggered = tree.arrays(event_level_keys, entry_stop=entry_stop)
events_triggered = events_untriggered[ak.numexpr.evaluate(trigger, events_untriggered)]

In [ ]:
nrows = len(events_untriggered.fields) // 4 + 1
fig, axs = plt.subplots(figsize=(12, 3 * nrows), ncols=4, nrows=nrows)
fig.suptitle(f"Event level info (trigger= {trigger})")
untriggered_kwargs = {"label": f"all ({len(events_untriggered.event)})"}
triggered_kwargs = {"label": f"triggered ({len(events_triggered.event)})"}
triggered_kwargs.update(dict(hatch="//", edgecolor="k", fill=False))
for var, ax in zip(events_untriggered.fields, axs.flatten()):
    if var[:4] in ["sum_", "nhit"]:
        ax.set_yscale("log")
        density = False
    else:
        ax.set_ylabel("normalized")
        density = True
    _, bins, _ = ax.hist(events_untriggered[var], density=density, **untriggered_kwargs)
    ax.hist(events_triggered[var], bins=bins, density=density, **triggered_kwargs)
    ax.set_title(var)
axs.flatten()[0].legend()
fig.tight_layout()
fig.savefig(img_path / "event_level_info.png", dpi=300)

In [ ]:
sef = SumEnergyFit(events_triggered)
sef.fig.savefig(img_path / "sum_energy.png", dpi=300)